### 1 -Importons les différentes librairies 

In [2]:
#!pip install tensorflow
import pandas as pd
import numpy as np
from datetime import datetime
import time
import warnings
warnings.filterwarnings("ignore")

In [5]:
#import tarfile
#file = tarfile.open("data/traffic_weather_ug16_june19.tar.gz", mode="r:gz")
#file.extractall()
#file.close()

### 2 - Importons le Dataset "TrafficWeatherEvent_Aug16_June19_Publish.csv"

In [2]:
tw_event = pd.read_csv("data/TrafficWeatherEvent_Aug16_June19_Publish.csv")

In [3]:
tw_event.tail()

,EventId,Source,Type,Severity,TMC,Description,StartTime(UTC),EndTime(UTC),TimeZone,LocationLat,LocationLng,Distance(mi),AirportCode,Number,Street,Side,City,County,State,ZipCode
25033252,T-21866308,T,Flow-Incident,NaN,214.0,Delays of six minutes on Santa Monica Fwy East...,2017-08-31 01:31:00,2017-08-31 04:14:04,US/Pacific,34.034252,-118.347931,5.83,KSMO,NaN,I-10 E,R,Los Angeles,Los Angeles,CA,90016.0
25033253,T-21866309,T,Congestion,Fast,74.0,Delays of two minutes on Golden State Fwy Nort...,2017-08-31 03:38:00,2017-08-31 04:21:08,US/Pacific,34.133957,-118.275269,3.60,KCQT,NaN,I-5 N,R,Los Angeles,Los Angeles,CA,90027.0
25033254,T-21866310,T,Flow-Incident,NaN,214.0,Delays of two minutes on Harbor Fwy Northbound...,2017-08-31 03:42:00,2017-08-31 04:23:05,US/Pacific,34.051392,-118.260674,2.67,KCQT,NaN,CA-110 S,R,Los Angeles,Los Angeles,CA,90017.0
25033255,T-21866311,T,Congestion,Slow,70.0,Severe delays of 25 minutes on Oceanside Blvd ...,2017-08-31 03:49:00,2017-08-31 04:30:12,US/Pacific,33.201733,-117.341331,0.78,KOKB,2689.0,Oceanside Blvd,R,Oceanside,San Diego,CA,92054.0
25033256,T-21866312,T,Congestion,Slow,71.0,Delays increasing and delays of four minutes o...,2017-08-31 03:27:00,2017-08-31 04:30:12,US/Pacific,34.110271,-118.336029,1.53,KBUR,2284.0,N Highland Ave,R,Los Angeles,Los Angeles,CA,90068.0


### Récupérons les coordonnées géographiques de chaque ville dans un dictionnaire afin d'afficher la carte

In [8]:
#geo_dic = {}
#for i in range(len(tw_event)):
#    cities = tw_event.City.iloc[i]
#    lat = tw_event.LocationLat.iloc[i]
#    lng = tw_event.LocationLng.iloc[i]
#    geo_dic["{}".format(cities)] = [lat, lng]

#### Choix max et min des latitudes et longitudes de chaque ville

In [9]:
#LosAngeles = df_cities[df_cities.City == "Los Angeles"]
#latL = LosAngeles.LocationLat
#lngL = LosAngeles.LocationLng
#np.min(latL), np.max(latL), np.min(lngL), np.max(lngL)

### 3 - 1 - Choix des villes sur le dataset

In [4]:
cities = {'LosAngeles': [33.700615, 34.353627, -118.683511, -118.074559], 
           'Houston': [29.497907,30.129003,-95.797178,-94.988191],
           'Austin': [30.079327, 30.596764,-97.968881,-97.504838],
           'Dallas': [32.559567,33.083278,-97.036586,-96.428928],
           'Charlotte': [34.970168,35.423667,-81.060925,-80.622687],
           'Atlanta': [33.612410,33.916999,-84.575600,-84.231911]}

time_zones = {'Houston':'US/Central', 'Charlotte':'US/Eastern', 'Dallas':'US/Central',
              'Atlanta':'US/Eastern', 'Austin':'US/Central', 'LosAngeles':'US/Pacific'}

# time interval to sample data for 
start = datetime(2018, 6, 1)
finish   = datetime(2018, 9, 2)

begin = datetime.strptime('2018-06-01 00:00:00', '%Y-%m-%d %H:%M:%S')
end   = datetime.strptime('2018-08-31 23:59:59', '%Y-%m-%d %H:%M:%S')

### 3 - 2 - Le Dataset du Trafic Event pour toutes les Villes Choisies

In [5]:
tw_event['StartTime(UTC)'] = tw_event['StartTime(UTC)'].astype('datetime64[ns]', errors = 'ignore')
tw_event['EndTime(UTC)'] = tw_event['EndTime(UTC)'].astype('datetime64[ns]', errors = 'ignore')

In [18]:
for c in cities:
    crds = cities[c]
    subset_all = tw_event[(tw_event['Source'] == 'T') & (tw_event['StartTime(UTC)'] >= start) & (tw_event['StartTime(UTC)'] < end) & 
                    (tw_event['LocationLat']>crds[0]) & (tw_event['LocationLat']<crds[1]) & (tw_event['LocationLng']>crds[2]) & 
                    (tw_event['LocationLng']<crds[3])& (tw_event["AirportCode"] != "KMWS")]
    
    subset_accidents = tw_event[(tw_event['Type']=='Accident') &  (tw_event['Source'] == 'T') &(tw_event['StartTime(UTC)'] >= start) & (tw_event['StartTime(UTC)'] < end) 
                          & (tw_event['LocationLat']>crds[0]) & (tw_event['LocationLat']<crds[1]) & (tw_event['LocationLng']>crds[2]) 
                          & (tw_event['LocationLng']<crds[3])]
    
    print("Pour {} nous avons {} incidents, dont {} accidents! un taux de {:.2f}".format(c, len(subset_all), len(subset_accidents), 
                                                                               len(subset_accidents)*1.0/len(subset_all)))
    
    subset_all.to_csv('data/temporary/TW_EVENT_{}_20180601_20180609.csv'.format(c), index=False)

Pour LosAngeles nous avons 90961 incidents, dont 8303 accidents! un taux de 0.09
Pour Houston nous avons 45463 incidents, dont 5988 accidents! un taux de 0.13
Pour Austin nous avons 20438 incidents, dont 4383 accidents! un taux de 0.21
Pour Dallas nous avons 30543 incidents, dont 3457 accidents! un taux de 0.11
Pour Charlotte nous avons 18592 incidents, dont 5604 accidents! un taux de 0.30
Pour Atlanta nous avons 23943 incidents, dont 2738 accidents! un taux de 0.11


### !!! Fin de generation de TW_Event_Cities, pour la suite, voir le fichier GenerateFeaturesVectors.ipynb  !!!